In [11]:
#bgr8转jpeg格式
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [12]:
#显示摄像头组件
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import time
# 线程功能操作库
import threading
import inspect
import ctypes
import HSV_Config

In [13]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [14]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)
display(image_widget)

Image(value=b'', format='jpeg', height='300', width='300')

In [15]:
image = cv2.VideoCapture(0)

image.set(3, 320)
image.set(4, 240)
image.set(5, 30)  #设置帧率
image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
image.set(cv2.CAP_PROP_BRIGHTNESS, 62) #设置亮度 -64 - 64  0.0
image.set(cv2.CAP_PROP_CONTRAST, 63) #设置对比度 -64 - 64  2.0
image.set(cv2.CAP_PROP_EXPOSURE, 4800) #设置曝光值 1.0 - 5000  156.0

ret, frame = image.read()
image_widget.value = bgr8_to_jpeg(frame)
update_hsv = HSV_Config.update_hsv()

In [16]:
color_hsv  = {"red"   : ((0, 70, 72), (7, 255, 255)),
              "green" : ((54, 109, 78), (77, 255, 255)),
              "blue"  : ((92, 100, 62), (121, 251, 255)),
              "yellow": ((26, 100, 91), (32, 255, 255))}

In [17]:
def TEST():
    global color_hsv
    while True:
        ret, frame = image.read()
        frame, binary = update_hsv.get_contours(frame,color_hsv)
        # 实时传回图像数据进行显示
        image_widget.value = bgr8_to_jpeg(frame)
        

In [18]:
thread1 = threading.Thread(target=TEST)
thread1.setDaemon(True)
thread1.start()

In [19]:
stop_thread(thread1)

In [20]:
image.release()              #使用完成对象记住释放掉对象，不然下一个程序使用这个对象模块会被占用，导致无法使用